In [ ]:
%load_ext autoreload
%autoreload 2
%pylab

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%run dqn.py

In [ ]:
F_HEIGHT = F_WIDTH = 84
preproc = Preprocessor(F_HEIGHT, F_WIDTH)
N = 3000
env = Env(preproc, 4, 4)

mem = ReplayMemory(N, F_HEIGHT, F_WIDTH, batch_size=32, frames_per_state=4)

rng = np.random.RandomState(42)
state = env.reset()
is_first_step = True
positive_rewards_to_store = 2
positive_rewards = 0
for i in range(N):
    action = rng.randint(2)
    ab = 2 if action == 0 else 3
    next_state, r, term = env.step(ab)
    
    round_done = (r != 0)  # for Pong
    if not round_done and  term:
        print "Warning!"
        
    if round_done and mem.term_[-1]:
        print i, "Repeat done!"
    mem.push(state, action, r, next_state, round_done, is_first_step)
    
    is_first_step = False

    if term:
        state = env.reset()
    else:
        state = next_state

    if round_done or term:
        is_first_step = True

    # learning
    if i % 500 == 0:
        print '\r{}'.format(i)
        
    if r>0:
        positive_rewards += 1
        print "Got r>0 {} time(s)".format(positive_rewards)
        if positive_rewards >= positive_rewards_to_store:
            break
    

In [ ]:
model = DQN((BATCH_SIZE, FRAMES_PER_STATE, F_HEIGHT, F_WIDTH))
model_ = DQN((BATCH_SIZE, FRAMES_PER_STATE, F_HEIGHT, F_WIDTH))
model.clone_to(model_)


**2017-07-08**

In the batch-processing script dqn.py, we had used a different set of parameters for RMSprop, could that be the problem? 

We didn't set eps and alpha in dqn.py, and set them when experimenting in the notebook.

### RMSProp's parameters matter!

In [ ]:
# loss_fn = torch.nn.SmoothL1Loss()
loss_fn = torch.nn.MSELoss()
optim_opts1 = {'lr':0.00025, 'momentum':MOMENTUM, 'eps':0.01, 'alpha':0.95}
optim_opts2 = {'lr':0.00025, 'momentum':MOMENTUM}

#optim = torch.optim.RMSprop(model.parameters(), **optim_opts2)
#tmp_dir = 'tmp3'

optim = torch.optim.RMSprop(model.parameters(), **optim_opts1)
tmp_dir = 'tmp2'

In [ ]:
manual_train_round = 0
while manual_train_round < 100:
    ### a bit training
    for i in range(100):
        l = learn_step(model, model_, mem, optim, loss_fn)
        if i%100 == 0: print "\r{}:{}".format(i, l),
    

    manual_train_round += 1

    ### evaluate
    pred_state_act_values, act, reward, target_vals = evaluate(model, model_, mem)
    pred_act_values = []
    for v, a in zip(pred_state_act_values, act):
        va = []
        for v_, a_ in zip(v, a):
            va.append([v_[0], v_[1], v_[a_],])
        pred_act_values.append(np.asarray(va))

    _ = model.clone_to(model_)
    ### plot
    episode_id = 0
    epis = range(len(pred_act_values))
    for episode_id in epis[-2:]:
        clf()
        plot(pred_act_values[episode_id][:,0], 'g:')
        plot(pred_act_values[episode_id][:,1], 'g--')
        plot(pred_act_values[episode_id][:,2], 'r')
        plot(reward[episode_id], 'b')
        plot(target_vals[episode_id], 'm--')
        savefig('{}/{}_{}.png'.format(tmp_dir, manual_train_round, episode_id))

In [ ]:
s0=rng.rand(4,8,10)

In [ ]:
f = rng.rand(8,10)

In [ ]:
s1 = np.concatenate((s0[1:], f[np.newaxis,...]))

In [ ]:
s1[0]==s0[1]

In [ ]:
for i_, t1_, t2_ in zip(range(len(mem)-1), mem.term_[:-1], mem.term_[1:]):
    if t1_ and t2_:
        print i_

In [ ]:
pred_act_values[0].max()

In [ ]:
for j in range(1000):
    state_batch, action_batch, reward_batch, \
        next_state_batch, term_batch = \
        mem.make_batch(32)
        
    rr = reward_batch.cpu().numpy()
    
    if np.sum(rr>0)>0:
        print np.nonzero(rr>0)

In [ ]:
model = DQN((BATCH_SIZE, FRAMES_PER_STATE, F_HEIGHT, F_WIDTH))

In [ ]:
x = Variable((mem.obs_[10].float()/255.0).unsqueeze(0), volatile=True)

In [ ]:
y = model(x)

In [ ]:
y.data[0].cpu().numpy()

In [ ]:
reward_batch.cpu().numpy()

In [ ]:
state_batch, action_batch, reward_batch, next_state_batch, term_batch = mem.make_batch(32)
batch_size=32
target_reward = torch.zeros(batch_size).type(FloatTensor)
pred_next_state_val = model(Variable(next_state_batch, volatile=True)).max(dim=1)[0] * 0.99
target_reward[...] = reward_batch
target_reward[1-term_batch] += pred_next_state_val.data[1-term_batch]

In [ ]:
lh = []
model = DQN((BATCH_SIZE, FRAMES_PER_STATE, F_HEIGHT, F_WIDTH))
model_ = DQN((BATCH_SIZE, FRAMES_PER_STATE, F_HEIGHT, F_WIDTH))
model.clone_to(model_)
loss_fn = torch.nn.MSELoss()
optim = torch.optim.RMSprop(model.parameters(), lr=1e-5, momentum=0.95)
for i in range(300000):
    ls = learn_step(model, model_, mem, optim, loss_fn)
    lh.append(ls)
    if i % 1000 == 0:
        print "\r",i, ls
        model.clone_to(model_)
L = np.asarray(lh).reshape(100,len(lh)/100)
# plot(L.mean(axis=1))
plot(lh)

In [ ]:
s = env.reset()

In [ ]:
policy.eps = 0.01

In [ ]:
policy(s)